<a href="https://colab.research.google.com/github/AdamClarkStandke/TinyMachineLearning/blob/main/ClassificationTFLuNano.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 2.x

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [2]:
import numpy as np
import pathlib
import tensorflow as tf
import tensorflow_datasets as tfds
import zipfile
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.models import Model

In [4]:
MODEL_ALPHA = 0.35
MODEL_INPUT_WIDTH = 48
MODEL_INPUT_HEIGHT = 48
TFL_MODEL_FILE = "recognize.tflite"
TFL_MODEL_HEADER_FILE = "recognize_model.h"
TF_MODEL = "recognize"

In [5]:
with zipfile.ZipFile("dataset.zip", 'r') as zip_ref:
    zip_ref.extractall(".")
data_dir = "dataset"

In [7]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  interpolation="bilinear",
  image_size=(MODEL_INPUT_WIDTH, MODEL_INPUT_HEIGHT)
  )

val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  interpolation="bilinear",
  image_size=(MODEL_INPUT_WIDTH, MODEL_INPUT_HEIGHT)
  )

Found 40 files belonging to 2 classes.
Using 32 files for training.
Found 40 files belonging to 2 classes.
Using 8 files for validation.


In [8]:
class_names = train_ds.class_names
num_classes = len(class_names)
print(class_names)

['bathroom', 'kitchen']


In [9]:
rescale = tf.keras.layers.Rescaling(1./255, offset= -1)
train_ds = train_ds.map(lambda x, y: (rescale(x), y))
val_ds   = val_ds.map(lambda x, y: (rescale(x), y))

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [10]:
base_model = MobileNetV2(input_shape=(MODEL_INPUT_WIDTH, MODEL_INPUT_HEIGHT, 3),
                         include_top=False,
                         weights='imagenet',
                         alpha=0.35)
base_model.trainable = False

feat_extr = base_model

2019640/2019640 [==============================] - 1s 1us/step


In [11]:
augmen = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
  tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal'),
])

train_ds = train_ds.map(lambda x, y: (augmen(x), y))
val_ds   = val_ds.map(lambda x, y: (augmen(x), y))

In [12]:
global_avg_layer = tf.keras.layers.GlobalAveragePooling2D()
dense_layer = tf.keras.layers.Dense(num_classes, activation='softmax')

In [13]:
inputs = tf.keras.Input(shape=(MODEL_INPUT_WIDTH, MODEL_INPUT_HEIGHT, 3))
x = global_avg_layer(feat_extr.layers[-1].output)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = dense_layer(x)
model = tf.keras.Model(inputs=feat_extr.inputs, outputs=outputs)

In [14]:
lr = 0.0005
model.compile(
  optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=False),
  metrics=['accuracy'])
     

In [15]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 48, 48, 3)]  0           []                               
                                                                                                  
 Conv1 (Conv2D)                 (None, 24, 24, 16)   432         ['input_1[0][0]']                
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 24, 24, 16)   64          ['Conv1[0][0]']                  
                                                                                                  
 Conv1_relu (ReLU)              (None, 24, 24, 16)   0           ['bn_Conv1[0][0]']               
                                                                                              

In [17]:
model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=100
)

Epoch 1/100
1/1 [==============================] - 1s 1s/step - loss: 0.5239 - accuracy: 0.7500 - val_loss: 1.0878 - val_accuracy: 0.3750
Epoch 2/100
1/1 [==============================] - 1s 699ms/step - loss: 0.4817 - accuracy: 0.7188 - val_loss: 1.3628 - val_accuracy: 0.3750
Epoch 3/100
1/1 [==============================] - 1s 953ms/step - loss: 0.4948 - accuracy: 0.7500 - val_loss: 0.8560 - val_accuracy: 0.6250
Epoch 4/100
1/1 [==============================] - 1s 854ms/step - loss: 0.5181 - accuracy: 0.7500 - val_loss: 1.1346 - val_accuracy: 0.2500
Epoch 5/100
1/1 [==============================] - 1s 673ms/step - loss: 0.3878 - accuracy: 0.8125 - val_loss: 1.3470 - val_accuracy: 0.2500
Epoch 6/100
1/1 [==============================] - 1s 659ms/step - loss: 0.6587 - accuracy: 0.6875 - val_loss: 1.0692 - val_accuracy: 0.7500
Epoch 7/100
1/1 [==============================] - 1s 1s/step - loss: 0.4736 - accuracy: 0.8125 - val_loss: 1.2770 - val_accuracy: 0.3750
Epoch 8/100
1/1 [==

In [18]:
model.save(TF_MODEL)

In [ ]:
TF_MODEL

# Quantize the TensorFlow model with the TFLite converter